# Get the data for single who wants to move to Austin

#### This is on assumption that he/she is looking for shopping nightlide and apartments

In [1]:
#define the Headers 
import folium
import pandas as pd
import numpy as np
import vincent

In [2]:
# Create the Data frame from night life 
nightlife_df = pd.read_csv('data_sources/nightlife_data_combined_with_cords.csv')
apartment_df = pd.read_csv('data_sources/Atx_apartment.csv')

nightlife_df = nightlife_df.drop(['Unnamed: 0'], axis=1)
apartment_df = apartment_df.drop(['Unnamed: 0'], axis=1)

nightlife_df.head()
apartment_df.head()

,apartment Name,Zip code,Rating,latitude,longitude
0,Polaris on the Park,78741,4.5,30.220450,-97.744565
1,Easton Hills Apartments,78741,4.5,30.234663,-97.731773
2,The Oaks,78704,4.5,30.236873,-97.757093
3,First Call Properties,78745,4.5,30.229017,-97.788482
4,Montecito,78741,2.0,30.223560,-97.741970


In [3]:
nightlife_df = nightlife_df.drop(['Club Count', 'Club Rating', "Total Nightlife Count", 'Combined Rating' ], axis=1)


In [4]:
nightlife_df.rename(columns={'Bar/Restaurant Count': 'Nightlife Count', 'Bar/Restaurant Rating': 'Nightlife Rating'}, inplace=True)

nightlife_df.head()

,ZipCode,Restaurant Count,Restaurant Rating,Bar Count,Bar Rating,Lat,Lon,Nightlife Count,Nightlife Rating
0,73301,3.0,4.500000,0.0,0.000000,30.220098,-97.749046,3.0,9.000000
1,78617,1.0,4.000000,2.0,3.000000,30.201016,-97.665245,3.0,6.666667
2,78646,2.0,4.500000,0.0,0.000000,30.273205,-97.800709,2.0,9.000000
3,78660,4.0,3.125000,2.0,3.000000,30.453564,-97.666576,6.0,6.166667
4,78701,201.0,4.186567,194.0,4.265464,30.267238,-97.741325,618.0,5.401294


In [5]:
count_df = pd.DataFrame(apartment_df.groupby('Zip code').count()['apartment Name'])
aparments_df = pd.DataFrame(apartment_df.groupby('Zip code').mean()['Rating'])
aparments_df['Apartment count']  = count_df['apartment Name']
aparments_df.rename(columns={"Rating": "apartment Rating"}, inplace=True)
aparments_df.head()

,apartment Rating,Apartment count
Zip code,,
78613,5.000000,1
78701,4.322034,118
78702,4.234375,96
78703,3.857143,49
78704,4.110390,77


### Merge the Data

In [46]:
df_single = pd.merge(nightlife_df, aparments_df, left_on='ZipCode', right_index=True, how='left' ).replace(np.nan, 0)
#df_school_house[list(["Zipcode", "homesalecount", 'gsRating'])] = df_school_house[list(["Zipcode",  "homesalecount", 'gsRating'])].astype(int)
df_single.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 11 columns):
ZipCode              51 non-null int64
Restaurant Count     51 non-null float64
Restaurant Rating    51 non-null float64
Bar Count            51 non-null float64
Bar Rating           51 non-null float64
Lat                  51 non-null float64
Lon                  51 non-null float64
Nightlife Count      51 non-null float64
Nightlife Rating     51 non-null float64
apartment Rating     51 non-null float64
Apartment count      51 non-null float64
dtypes: float64(10), int64(1)
memory usage: 4.5 KB


## Plot the map

In [51]:
def Plotmarkers(mp, df):
    index = ['Restaurants', 'Bars', 'Apartment']
    
    for i, row in df.iterrows():
        #get the bars that needs to be plotted 
        data = {}
        #Generalize the rating to base 10
        data['Rating'] = [df.loc[i]['Restaurant Rating']*2, df.loc[i]['Bar Rating']*2,
                          df.loc[i]['apartment Rating']*2] 
        
        #Convert the dict into DataFrame
        df_bar = pd.DataFrame(data, index=index)
        
        #Plot a Barchart
        bar_chart = vincent.GroupedBar(df_bar, width=100, height=100 )
        
        #define color x and y axis 
        bar_chart.colors(range_= [ '#8856a7'])        
        x = df.loc[i]["ZipCode"]
        bar_chart.axis_titles(x="Night Life", y='Ratings')
        
        #tilt the y axis
        ax = vincent.AxisProperties(labels = vincent.PropertySet(angle=vincent.ValueRef(value=-45)))
        bar_chart.axes[0].properties = ax
        
        #create a pop up
        popup = folium.Popup()
        folium.Vega(bar_chart, height=150, width=200).add_to(popup)
        #add to the marker
        folium.Marker([df.loc[i]['Lat'], df.loc[i]['Lon']], popup=popup,
                 tooltip=df.loc[i]["ZipCode"].astype(int)).add_to(mp) 

    return mp

In [52]:
def Plotmap(mp, df):
    
    # Add marker one by one on the map
    for i, row in df.iterrows():
        #print(f"index {i}")
        folium.Circle(
            location=[df.loc[i]['Lat'], df.loc[i]['Lon']],
            popup=str(df.loc[i]['ZipCode']),
            radius=(df.loc[i]['Nightlife Count']+df.loc[i]['Apartment count'])*5,
            color='crimson',
            fill=True,
            fill_color='crimson'
        ).add_to(mp)  
        
    Plotmarkers(mp, df)
    return mp 
#Plotmap(nightlife_df)


## Call the functions to plot the graph

In [54]:
#define Austin map Lat Lon[30.35, -97.75]
mp = folium.Map(location=[30.35, -97.75], zoom_start=11)

#choice could be anything for this notebook it is only Nightlife
choice = 'Nightlife'
if choice == 'Nightlife':
    Plotmap(mp, df_single)
else:
    print("Nightlife")
    
mp.save('SingleChart.html')
mp
    